Successfully loaded training_args.bin: TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStra

C:\Users\ahmad\AppData\Local\Temp\ipykernel_2320\2713772912.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_training_args = torch.load(training_args_path)


In [15]:
## Dear MR, I would not try this
"""import torch
import torch
import os

LORA_CHECKPOINT_DIR = r"../checkpoint-290/"

training_args_path = os.path.join(LORA_CHECKPOINT_DIR, "training_args.bin")

if os.path.exists(training_args_path):
    # Load the TrainingArguments object
    # It's usually a serialized object
    loaded_training_args = torch.load(training_args_path)
    print(f"Successfully loaded training_args.bin: {loaded_training_args}")
    # You can access its attributes, e.g.,
    # print(f"Per device train batch size: {loaded_training_args.per_device_train_batch_size}")
    # print(f"Learning rate: {loaded_training_args.learning_rate}")
else:
    print(f"Error: training_args.bin not found at {training_args_path}")
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import os

LORA_CHECKPOINT_DIR = r"D:\Industry\Projects\machine learning chatbot\checkpoint-290"
BASE_MODEL_NAME = "microsoft/Phi-4-mini-instruct"

# --- NEW: Define a directory for disk offloading ---
# Make sure this directory exists and has enough free space (tens of GBs potentially)
OFFLOAD_CACHE_DIR = r"D:\Industry\Projects\machine learning chatbot\offload_cache"
os.makedirs(OFFLOAD_CACHE_DIR, exist_ok=True) # Ensure the directory exists

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    llm_int8_enable_fp32_cpu_offload=True
)

print(f"Attempting to load model with explicit device_map and disk offloading: {BASE_MODEL_NAME}...")

# Manually create a device map to force modules to CPU if GPU is full.
# If even CPU runs out, it will now use the offload_folder.
device_map = {
    "model.embed_tokens": 0,
    "model.norm": 0,
    "lm_head": 0,
}

num_layers = 32
layers_on_gpu = 8 # <<<<<<<<<<<<< Keep experimenting with this if you want to optimize GPU usage

for i in range(num_layers):
    layer_name = f"model.layers.{i}"
    if i < layers_on_gpu:
        device_map[layer_name] = 0 # Try to put on GPU 0
    else:
        device_map[layer_name] = "cpu" # Else, put on CPU

print(f"Constructed device_map: {device_map}")

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    quantization_config=bnb_config,
    device_map=device_map,
    low_cpu_mem_usage=True, # Helps with loading large models
    offload_folder=OFFLOAD_CACHE_DIR, # Specify where to offload to disk
    trust_remote_code=True
)
print("Base model loaded with explicit device_map and disk offloading enabled.")

# --- Continue with your existing code for LoRA and tokenizer ---
if os.path.isdir(LORA_CHECKPOINT_DIR):
    model = PeftModel.from_pretrained(model, LORA_CHECKPOINT_DIR)
    print("LoRA adapters loaded.")
    model = model.merge_and_unload()
    print("LoRA adapters merged.")
else:
    print(f"Warning: LoRA checkpoint directory '{LORA_CHECKPOINT_DIR}' not found. Loading base model only.")

model.eval()
print("Model set to evaluation mode.")

tokenizer = AutoTokenizer.from_pretrained(LORA_CHECKPOINT_DIR, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
print("Tokenizer loaded.")

print("\n!!! WARNING: Inference will be VERY SLOW due to heavy disk offloading !!!\n")
print("Consider a smaller model or GGUF for better performance on limited hardware.")

def generate_response(question, max_new_tokens=150, temperature=0.7, top_p=0.9, top_k=50):
    # 1. Prepare your input prompt:
    # This must match the format you used during fine-tuning (Q: question\nA: answer)
    prompt = f"Q: {question}\nA:"

    # 2. Tokenize the input
    # return_tensors="pt" ensures PyTorch tensors are returned.
    # .to(model.device) moves the tensors to the GPU if the model is on GPU.
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

    # 3. Generate text
    print(f"Generating response for: '{question}'...")
    with torch.no_grad(): # Essential: Disable gradient calculations during inference to save memory and speed up
        outputs = model.generate(
            input_ids=input_ids,
            max_new_tokens=max_new_tokens,
            num_return_sequences=1,
            do_sample=True,      # Enable sampling (crucial for temperature/top_p/top_k to have an effect)
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            eos_token_id=tokenizer.eos_token_id, # Stop when EOS token is generated
            pad_token_id=tokenizer.pad_token_id, # Use pad token during generation if context varies
            attention_mask=input_ids.ne(tokenizer.pad_token_id).long() # Provide attention mask
        )

    # 4. Decode the output
    # outputs[0] gives the first (and only) generated sequence.
    # [len(input_ids[0]):] slices to get *only* the newly generated tokens, not the prompt.
    # skip_special_tokens=True removes tokens like <s>, </s>, <pad> from the output.
    response = tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True)

    # Clean up any potential extra newline/space characters
    return response.strip()

### --- Let's do some inference! ---

# Example 1: A question likely covered in your Q&A fine-tuning
print("-" * 30)
q = "What is the primary function of a transformer in deep learning?"
a = generate_response(q)
print(f"Question: {q}\nAnswer: {a}\n")
print("-" * 30)
q = "What is the your purpose?"
a = generate_response(q)
print(f"Question: {q}\nAnswer: {a}\n")"""

Attempting to load model with explicit device_map and disk offloading: microsoft/Phi-4-mini-instruct...
Constructed device_map: {'model.embed_tokens': 0, 'model.norm': 0, 'lm_head': 0, 'model.layers.0': 0, 'model.layers.1': 0, 'model.layers.2': 0, 'model.layers.3': 0, 'model.layers.4': 0, 'model.layers.5': 0, 'model.layers.6': 0, 'model.layers.7': 0, 'model.layers.8': 'cpu', 'model.layers.9': 'cpu', 'model.layers.10': 'cpu', 'model.layers.11': 'cpu', 'model.layers.12': 'cpu', 'model.layers.13': 'cpu', 'model.layers.14': 'cpu', 'model.layers.15': 'cpu', 'model.layers.16': 'cpu', 'model.layers.17': 'cpu', 'model.layers.18': 'cpu', 'model.layers.19': 'cpu', 'model.layers.20': 'cpu', 'model.layers.21': 'cpu', 'model.layers.22': 'cpu', 'model.layers.23': 'cpu', 'model.layers.24': 'cpu', 'model.layers.25': 'cpu', 'model.layers.26': 'cpu', 'model.layers.27': 'cpu', 'model.layers.28': 'cpu', 'model.layers.29': 'cpu', 'model.layers.30': 'cpu', 'model.layers.31': 'cpu'}


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


Base model loaded with explicit device_map and disk offloading enabled.


ValueError: You are trying to offload the whole model to the disk. Please use the `disk_offload` function instead.

In [3]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()  # loads .env file

API_TOKEN = os.getenv("Hugging_face_token")
MODEL_ID = "mlahmadmostafa/zaboolamlchatbot"


headers = {
    "Authorization": f"Bearer {API_TOKEN}"
}

data = {
    "inputs": "Hello, how are you?"
}

response = requests.post(
    f"https://api-inference.huggingface.co/models/{MODEL_ID}",
    headers=headers,
    json=data
)
print(response.status_code)
print(response.text)
print(response.json())

404
Not Found


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [8]:
from huggingface_hub import HfApi

api = HfApi()
user_info = api.whoami(token=API_TOKEN)
print(user_info)


{'type': 'user', 'id': '66f6f0d12fd4a6f4be6b322c', 'name': 'mlahmadmostafa', 'fullname': 'Ahmad Mostafa', 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/66f6f0d12fd4a6f4be6b322c/zsRVRyTp-gRerfRHFelay.jpeg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'zaboolamlchatbot', 'role': 'fineGrained', 'createdAt': '2025-06-08T08:34:39.177Z', 'fineGrained': {'canReadGatedRepos': True, 'global': ['discussion.write', 'post.write'], 'scoped': [{'entity': {'_id': '68454ac6a02690b5bce1fe7f', 'type': 'model', 'name': 'mlahmadmostafa/zaboolamlchatbot'}, 'permissions': ['repo.content.read', 'discussion.write', 'repo.write']}, {'entity': {'_id': '66f6f0d12fd4a6f4be6b322c', 'type': 'user', 'name': 'mlahmadmostafa'}, 'permissions': ['repo.content.read', 'repo.write', 'inference.serverless.write', 'inference.endpoints.infer.write', 'inference.endpoints.write', 'user.webhooks.read', 'user.webhooks.writ

In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM

# 1. Load base model in 4-bit
base_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-4-mini-instruct",
    load_in_4bit=True,
    device_map="auto",
    trust_remote_code=True
)
lora_model = PeftModel.from_pretrained(base_model, "../checkpoint-290/")

# Merge LoRA weights into base model
merged_model = lora_model.merge_and_unload()
merged_model.save_pretrained("../Model/The_Merged_Model")

AttributeError: partially initialized module 'torch' has no attribute 'version' (most likely due to a circular import)